In [1]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.5 MB/s eta 0:00:00


# Import Libraries

In [2]:
from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

import tempfile
import os

# Load the Dataset

In [3]:
df_ratings = pd.read_csv('https://raw.githubusercontent.com/ameliaputrif/Capstone-Project-CH2-PS451/main/ML/dataset/tourism_rating_user.csv')
df_places = pd.read_csv('https://raw.githubusercontent.com/ameliaputrif/Capstone-Project-CH2-PS451/main/ML/dataset/tourism_with_id.csv')

In [4]:
df_ratings.head(10)

,User_Id,Location,Age,Gender,Generation,Place_Name,Place_Id,Place_Ratings,Description,Category,City,Price,Price_Category
0,1,"Semarang, Jawa Tengah",20,Laki-laki,Gen-Z,Candi Ratu Boko,179,3,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
1,22,"Subang, Jawa Barat",25,Perempuan,Gen-Z,Candi Ratu Boko,179,4,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
2,40,"Semarang, Jawa Tengah",27,Laki-laki,Milenial,Candi Ratu Boko,179,3,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
3,49,"Jakarta Pusat, DKI Jakarta",24,Laki-laki,Gen-Z,Candi Ratu Boko,179,5,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
4,74,"Semarang, Jawa Tengah",30,Laki-laki,Milenial,Candi Ratu Boko,179,3,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
5,76,"Cirebon, Jawa Barat",19,Perempuan,Gen-Z,Candi Ratu Boko,179,1,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
6,84,"Yogyakarta, DIY",35,Laki-laki,Milenial,Candi Ratu Boko,179,4,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
7,127,"Serang, Banten",38,Laki-laki,Milenial,Candi Ratu Boko,179,4,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
8,127,"Serang, Banten",38,Laki-laki,Milenial,Candi Ratu Boko,179,3,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang
9,134,"Nganjuk, Jawa Timur",27,Perempuan,Milenial,Candi Ratu Boko,179,2,Situs Ratu Baka atau Candi Boko (Hanacaraka:ꦕꦤ...,Budaya,Yogyakarta,75000.0,Sedang


In [5]:
df_places = df_places.drop(columns=['Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Unnamed: 11', 'Unnamed: 12'])
df_places.head(10)

,Place_Id,Place_Name,Description,Category,City,Price,Rating
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5
5,6,Taman Impian Jaya Ancol,Taman Impian Jaya Ancol merupakan sebuah objek...,Taman Hiburan,Jakarta,25000,4.5
6,7,Kebun Binatang Ragunan,Kebun Binatang Ragunan adalah sebuah kebun bin...,Cagar Alam,Jakarta,4000,4.5
7,8,Ocean Ecopark,Ocean Ecopark Salah satu zona rekreasi Ancol y...,Taman Hiburan,Jakarta,180000,4.0
8,9,Pelabuhan Marina,Pelabuhan Marina Ancol berada di kawasan Taman...,Bahari,Jakarta,175000,4.4
9,10,Pulau Tidung,Pulau Tidung adalah salah satu kelurahan di ke...,Bahari,Jakarta,150000,4.5


Convert a pandas data frame to a TensorFlow dataset

In [6]:
tensor_ratings = tf.data.Dataset.from_tensor_slices({
    'User_Id': tf.cast(df_ratings['User_Id'].values, dtype=tf.int64),
    'Location': tf.convert_to_tensor(df_ratings['Location'].values, dtype=tf.string),
    'Age': tf.cast(df_ratings['Age'].values, tf.int64),
    'Gender': tf.cast(df_ratings['Gender'].values, dtype=tf.string),
    'Generation': tf.cast(df_ratings['Generation'].values, dtype=tf.string),
    'Place_Name': tf.convert_to_tensor(df_ratings['Place_Name'].values, dtype=tf.string),
    'Place_Id': tf.cast(df_ratings['Place_Id'].values, tf.int64),
    'Place_Ratings': tf.cast(df_ratings['Place_Ratings'].values, tf.int64),
    'Description': tf.convert_to_tensor(df_ratings['Description'].values, dtype=tf.string),
    'Category': tf.convert_to_tensor(df_ratings['Category'].values, dtype=tf.string),
    'City': tf.convert_to_tensor(df_ratings['City'].values, dtype=tf.string),
    'Price': tf.cast(df_ratings['Price'].values, tf.int64),
    'Price_Category': tf.cast(df_ratings['Price_Category'].values, dtype=tf.string),
})

tensor_places = tf.data.Dataset.from_tensor_slices({
    'Place_Id': tf.cast(df_places['Place_Id'].values, tf.int64),
    'Place_Name': tf.convert_to_tensor(df_places['Place_Name'].values, dtype=tf.string),
    'Description': tf.convert_to_tensor(df_places['Description'].values, dtype=tf.string),
    'Category': tf.convert_to_tensor(df_places['Category'].values, dtype=tf.string),
    'City': tf.convert_to_tensor(df_places['City'].values, dtype=tf.string),
    'Price': tf.cast(df_places['Price'].values, tf.int64),
    'Rating': tf.cast(df_places['Rating'].values, tf.float64),
})

In [7]:
ratings = tensor_ratings.map(lambda x: {
    "Place_Name": x["Place_Name"],
    'Generation': x["Generation"],
    'Gender': x['Gender'],
    'City': x['City'],
    'Category': x['Category'],
    'Price_Category': x['Price_Category']
})

places = tensor_places.map(lambda x: x["Place_Name"])

# Users and Places Feature

For Users:
* Generation
* Gender
* City
* Category
* Price Category

For Places:
* Place Name

In [8]:
unique_generation = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Generation"]))))

unique_gender = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Gender"]))))

unique_city = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["City"]))))

unique_category = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Category"]))))

unique_price = np.unique(np.concatenate(list(ratings.batch(64).map(lambda x: x["Price_Category"]))))

unique_place_name = np.unique(np.concatenate(list(places.batch(64))))

# Build the Models

## User Model

In [9]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.generation_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_generation, mask_token=None),
        tf.keras.layers.Embedding(len(unique_generation) + 1, 32),
    ])

    self.gender_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_gender, mask_token=None),
        tf.keras.layers.Embedding(len(unique_gender) + 1, 32),
    ])

    self.city_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_city, mask_token=None),
        tf.keras.layers.Embedding(len(unique_city) + 1, 32),
    ])

    self.category_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_category, mask_token=None),
        tf.keras.layers.Embedding(len(unique_category) + 1, 32),
    ])

    self.price_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_price, mask_token=None),
        tf.keras.layers.Embedding(len(unique_price) + 1, 32),
    ])

  def call(self, inputs):
    return tf.concat([
        self.generation_embedding(inputs["Generation"]),
        self.gender_embedding(inputs['Gender']),
        self.city_embedding(inputs["City"]),
        self.category_embedding(inputs["Category"]),
        self.price_embedding(inputs["Price_Category"]),
    ], axis=1)

## Query Model

In [10]:
class QueryModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = UserModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

## Place Model

In [11]:
class PlaceModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10000

    self.place_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_place_name,mask_token=None),
      tf.keras.layers.Embedding(len(unique_place_name) + 1, 32)
    ])

    self.place_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.place_text_embedding = tf.keras.Sequential([
      self.place_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.place_vectorizer.adapt(places)

  def call(self, place):
    return tf.concat([
        self.place_embedding(place),
        self.place_text_embedding(place),
    ], axis=1)

## Candidate Model

In [12]:
class CandidateModel(tf.keras.Model):

  def __init__(self, layer_sizes):
    super().__init__()

    self.embedding_model = PlaceModel()

    self.dense_layers = tf.keras.Sequential()

    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

## Place Destination Model

In [13]:
class PlaceDestinationModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=places.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):

    query_embeddings = self.query_model({
        "Generation": features["Generation"],
        'Gender': features['Gender'],
        "City": features["City"],
        "Category": features["Category"],
        "Price_Category": features["Price_Category"],
    })
    place_embeddings = self.candidate_model(features["Place_Name"])

    return self.task(
        query_embeddings, place_embeddings, compute_metrics=not training)

# Training the Model

Split the data into training and test datasets (80-20)

In [14]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(len(ratings), seed=42, reshuffle_each_iteration=False)

train = shuffled.take(int(0.8 * len(ratings)))
test = shuffled.skip(int(0.8 * len(ratings))).take(int(0.2 * len(ratings)))

cached_train = train.shuffle(len(ratings)).batch(256)
cached_test = test.batch(512).cache()

In [15]:
num_epochs = 50

model = PlaceDestinationModel([64])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

Top-100 accuracy: 1.00.


In [16]:
model.evaluate(cached_test, return_dict=True)

4/4 [==============================] - 1s 126ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0651 - factorized_top_k/top_5_categorical_accuracy: 0.3318 - factorized_top_k/top_10_categorical_accuracy: 0.5038 - factorized_top_k/top_50_categorical_accuracy: 0.9975 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 1497.3703 - regularization_loss: 0.0000e+00 - total_loss: 1497.3703


{'factorized_top_k/top_1_categorical_accuracy': 0.06513994932174683,
 'factorized_top_k/top_5_categorical_accuracy': 0.33180662989616394,
 'factorized_top_k/top_10_categorical_accuracy': 0.5038167834281921,
 'factorized_top_k/top_50_categorical_accuracy': 0.9974554777145386,
 'factorized_top_k/top_100_categorical_accuracy': 1.0,
 'loss': 1293.7235107421875,
 'regularization_loss': 0,
 'total_loss': 1293.7235107421875}

# Recommendation Result



In [17]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)
index.index_from_dataset(
  tf.data.Dataset.zip((places.batch(128), places.batch(128).map(model.candidate_model)))
)


_, place = index({
    "Generation": np.array(['Gen-Z']),
    'Gender': np.array(['Laki-laki']),
    "City": np.array(['Jakarta']),
    "Category": np.array(['Taman Hiburan']),
    "Price_Category": np.array(['Sedang'])},
    k=50
)


In [18]:
place[0].numpy()

array([b'The Escape Hunt', b'Skyrink - Mall Taman Anggrek',
       b'Atlantis Water Adventure', b'Sea World', b'Saloka Theme Park',
       b'Museum Macan (Modern and Contemporary Art in Nusantara)',
       b'Water Blaster Bukit Candi Golf',
       b'Jogja Bay Pirates Adventure Waterpark', b'Pulau Pari',
       b'Pulau Tidung', b'Panghegar Waterboom Bandung',
       b'Atlantis Land Surabaya', b'Ciputra Waterpark',
       b'Batununggal Indah Club', b'Tektona Waterpark',
       b'Taman Lapangan Banteng', b'Jakarta Planetarium',
       b'Taman Impian Jaya Ancol', b'Waterboom PIK (Pantai Indah Kapuk)',
       b'Taman Ayodya', b'Upside Down World Bandung',
       b'Wisata Agro Edukatif Istana Susu Cibugary', b'Taman Suropati',
       b'Taman Mini Indonesia Indah (TMII)', b'Museum Ullen Sentalu',
       b'Taman Menteng', b'Ocean Ecopark', b'Taman Agrowisata Cilangkap',
       b'Taman Legenda Keong Emas', b'Taman Cattleya', b'Candi Ratu Boko',
       b'Hutan Kota Srengseng', b'Umbul Sidomukti'

# Save and Load the Model

In [19]:
with tempfile.TemporaryDirectory() as tmp:
  path = os.path.join(tmp, "model")

  tf.saved_model.save(index, path)

  loaded = tf.saved_model.load(path)

  _, place = loaded({
    "Generation": np.array(['Milenial']),
    'Gender': np.array(['Laki-laki']),
    "City": np.array(['Bandung']),
    "Category": np.array(['Budaya']),
    "Price_Category": np.array(['Murah'])}
)

  print(f"Top 3 Recommendations: {place[0][:3]}")

Top 3 Recommendations: [b'Taman Budaya Jawa Barat' b'Museum Gedung Sate' b'Taman Sejarah Bandung']


In [20]:
place_strings = [place.decode('utf-8') for place in place.numpy()[0]]
place_recommendation = place_strings[:3]
print(f"Top 3 Recommendations: {place_recommendation}")

Top 3 Recommendations: ['Taman Budaya Jawa Barat', 'Museum Gedung Sate', 'Taman Sejarah Bandung']


In [21]:
top_recommendations = place_recommendation
top_data = df_places[df_places['Place_Name'].isin(top_recommendations)]

print('Top 3 Recommendation\n')
for index, row in top_data.iterrows():
    print(f"Place_Name: {row['Place_Name']}")
    print(f"Rating: {row['Rating']}")
    print(f"Description: {row['Description']}")
    print(f"Price: {row['Price']}\n")

Top 3 Recommendation

Place_Name: Museum Gedung Sate
Rating: 4.6
Description: Salah satu museum yang menjadi ikon kota ini adalah Museum Gedung Sate. Museum yang berada di gedung yang sama dengan kantor dinas Gubernur Jawa Barat ini dibangun pada masa kolonial oleh arsitek Belanda, Johan Gerber. Diresmikan pada 8 Desember 2017 dan mengusung 'Smart Museum', Museum Gedung Sate ini menjadi museum interaktif pertama di Indonesia. Konsep tersebut memberikan panduan informasi mengenai bangunan bersejarah menggunakan teknologi digital.
Price: 5000

Place_Name: Taman Budaya Jawa Barat
Rating: 4.3
Description: Taman Budaya Jawa Barat (TBJB) didirikan pada tahun 1991 di Kawasan Dago Tea House. TBJB memiliki fasilitas gedung teater tertutup, teater terbuka, ruang pameran, sekertariat, sanggar olah seni, wisma seni, dan cafetaria. Semua fasilitas tersebut diperuntunkan untuk kelancaran beraktifitas dan berkarya para warga Bandung, khususnya para seniman. TBJB menghelat berbagai macam agenda yang m